# Task 2 Encoding and Classifier

## Problem 
<br>
Encoding the following files in a quantum circuit mock_train_set.csv and mock_test_set.csv in at least two different ways (these could be basis, angle, amplitude, kernel or random encoding
<br>
<br>
● Design a variational quantum circuit for each of the encodings, uses the column 4 as the target, this is a binary class 0 and 1.<br>
● You must use the data from column0 to column3 for your proposed classifier.<br>
● Consider the ansatz you are going to design as a layer and find out how many layers are
necessary to reach the best performance. <br>

### Analyze and discuss the results

Feel free to use existing frameworks (e.g. PennyLane, Qiskit) for creating and training the circuits.<br><br>
This PennyLane demo can be useful: Training a quantum circuit with Pytorch,<br>
This Quantum Tensorflow tutorial can be useful: Training a quantum circuit with Tensorflow.<br>
For the variational circuit, you can try any circuit you want. You can start from one with a layer of RX, RZ and CNOTs.<br>
<br>
## References
* https://pennylane.ai/qml/demos/tutorial_state_preparation.html
* https://www.tensorflow.org/quantum/tutorials/mnist

<br>

## Clasification Using Amplitude Encoding on 2 Qubits

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from ipywidgets import widgets
from IPython.display import display, HTML

In [2]:
import pennylane as qml
from pennylane import numpy as np
from pennylane.optimize import NesterovMomentumOptimizer

import sys
from math import sqrt, pi

import pandas as pd
import scipy
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer
# supress a warning that is not useful here
pd.options.mode.chained_assignment = None


In [3]:
# sklearn StandardScaler
# https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html

path_train = '/_jupyter/QC/QOSF-challenge-md-2022/task-02/mock_train_set.csv'
path_test = '/_jupyter/QC/QOSF-challenge-md-2022/task-02/mock_test_set.csv'

df = pd.read_csv(path_train)
df_c = df.copy(deep=True)
df['1'] = np.log10(df_c['1'])
df['2'] = np.log10(df_c['2'])

f = lambda x: -1.0 if x==0 else 1.0
df['4'] = df_c['4'].map(f)

# npdf = df2.to_numpy()
npdf = df.to_numpy()
data = np.array(npdf)

print("Train data standardised:\n", data)

X = data[:, 0:4]
Y = data[:, -1]

# scale the data using sklearn StandardScaler
# std_slc = StandardScaler(with_mean=False)
std_slc = StandardScaler(with_mean=True)
std_slc.fit(X)
X_std = std_slc.transform(X)

# normalize data using sklearn StandardScaler
normalizer = Normalizer().fit(X_std)  # fit does nothing.
X_norm = normalizer.transform(X_std)

# features will be the angles vector
features = np.array(X_norm, requires_grad=False)
print("Train data normalized:\n", features)




Train data standardised:
 [[ 2.78926e+03  3.00000e+00  1.00000e+00  2.00000e+01 -1.00000e+00]
 [ 4.04001e+03  6.00000e+00  0.00000e+00  1.00000e+00  1.00000e+00]
 [ 2.93120e+03  4.00000e+00  4.00000e+00  4.00000e+01  1.00000e+00]
 ...
 [ 4.18281e+03  0.00000e+00  0.00000e+00  6.50000e+01 -1.00000e+00]
 [ 3.11375e+03  4.00000e+00  2.00000e+00  1.00000e+00  1.00000e+00]
 [ 4.56757e+03  4.00000e+00  5.00000e+00  9.00000e+01  1.00000e+00]]


StandardScaler()

Train data normalized:
 [[ 0.08683356 -0.03797654 -0.66528081 -0.74055328]
 [ 0.37612267  0.45694046 -0.51293519 -0.62179952]
 [ 0.26356815  0.46027486  0.79974818 -0.28121474]
 ...
 [ 0.47981546 -0.57210908 -0.59413647  0.29911565]
 [ 0.1940783   0.21079313 -0.20305745 -0.93630526]
 [ 0.56534373  0.1530959   0.47275529  0.65836961]]


Circuit coding for angle encoding follows pennylane technique, whch is also following the scheme in in Schuld and Petruccione (2018).<br>
""We had to also decompose controlled Y-axis rotations into more basic circuits following Nielsen and Chuang (2010).""<br>

* https://link.springer.com/book/10.1007/978-3-319-96424-9
* 

In [4]:
num_qubits = 4
num_layers = 8

dev = qml.device("default.gaussian", wires=num_qubits)

### test angle encoding

In [5]:
# trainable circuit layer
def layer(W):
    for i in range(num_qubits):

        mag_alpha =  W[i, 0]
        phase_alpha = W[i, 1]
        phi = W[i, 2]

        qml.Displacement(mag_alpha, phase_alpha, wires=i)
        qml.Rotation(phi, wires=i)

# circuit that gets evaluated and optimised on each step
@qml.qnode(dev)
def circuit(weights, data):

    qml.DisplacementEmbedding(features=data, wires=range(num_qubits), method='phase', c=0.5)
    # qml.DisplacementEmbedding(features=data, wires=range(num_qubits), method='amplitude')

    # apply trainable layers
    for W in weights:
        layer(W)

    # return qml.expval(qml.PauliZ(0))
    return qml.expval(qml.NumberOperator(wires=1))

draw_flag = 0
def variational_classifier(weights, bias, data):
    
    global draw_flag

    if draw_flag:
        draw_flag=0
        # qml.draw

    return circuit(weights, data) + bias

# ###############################################################################
# standard square loss
def square_loss(labels, predictions):
    
    loss = 0
    for l, p in zip(labels, predictions):
        loss = loss + (l - p) ** 2

    loss = loss / len(labels)
    return loss

# ###############################################################################
# goal: maximize accuracy
def accuracy(labels, predictions):

    loss = 0
    for l, p in zip(labels, predictions):
        if abs(l - p) < 1e-5:
            loss = loss + 1
    loss = loss / len(labels)

    return loss


def cost(weights, bias, features, labels):
    
    predictions = [variational_classifier(weights, bias, f) for f in features]
    return square_loss(labels, predictions)


In [6]:
np.random.seed(0)
num_data = len(Y)

# num_train = int(0.75 * num_data)
num_train = int(0.75 * num_data)
index = np.random.permutation(range(num_data))

feats_train = features[index[:num_train]]
Y_train = Y[index[:num_train]]

feats_val = features[index[num_train:]]
Y_val = Y[index[num_train:]]

# We need these later for plotting
X_train = X[index[:num_train]]
X_val = X[index[num_train:]]

# ######################################################################
# Load the test dataset and apply same transformations as we did with the train dataset.
df_test = pd.read_csv(path_test)
df_test_c = df_test.copy(deep=True)
df_test['1'] = np.log10(df_test_c['1'])
df_test['2'] = np.log10(df_test_c['2'])

f = lambda x: -1.0 if x==0 else 1.0
df_test['4'] = df_test_c['4'].map(f)
data_test = df_test.to_numpy()

X_test_ini = data_test[:, 0:4]
Y_test = data_test[:, -1]

# scale data using sklearn StandardScaler
std_slc = StandardScaler(with_mean=False)
std_slc.fit(X_test_ini)
X_test_std = std_slc.transform(X_test_ini)

# normalize data using sklearn StandardScaler
normalizer = Normalizer().fit(X_test_std)  # fit does nothing.
X_test_norm = normalizer.transform(X_test_std)

# convert to a pennylane numpy array
X_test = np.array(X_test_norm, requires_grad=False)


# ######################################################################
# accuracy for test dtaset
def test_accuracy(weights, bias):
    # apply the variational clasifier circuit on test dataset
    # using the learned weights
    predictions_test = [np.sign(variational_classifier(weights, bias, f)) for f in X_test]

    return accuracy(Y_test, predictions_test)




StandardScaler(with_mean=False)

In [7]:
weights_init = 0.01 * np.random.randn(num_layers, num_qubits, 3, requires_grad=True)
bias_init = np.array(0.0, requires_grad=True)

# initialise the optimizer
# opt = qml.GradientDescentOptimizer(stepsize=0.05)
opt = NesterovMomentumOptimizer(0.01)
# opt = NesterovMomentumOptimizer(0.008)
# opt = NesterovMomentumOptimizer(0.1)
# opt = NesterovMomentumOptimizer(0.1)
# batch_size = 5
# batch_size = 15
# batch_size = 20
batch_size = 10


# train the variational classifier
weights = weights_init
bias = bias_init

# steps = 50 # acc: 0.94
# steps = 149  # acc: 0.92
# steps = 150  # acc: 0.92
# steps = 42  # acc: 0.85

# phase
steps = 95 # acc:
# steps = 72 # acc:0.90


for it in range(steps):

    # Update the weights by one optimizer step
    batch_index = np.random.randint(0, num_train, (batch_size,))
    feats_train_batch = feats_train[batch_index]
    Y_train_batch = Y_train[batch_index]
    weights, bias, _, _ = opt.step(cost, weights, bias, feats_train_batch, Y_train_batch)

    # Compute predictions on train and validation set
    predictions_train = [np.sign(variational_classifier(weights, bias, f)) for f in feats_train]
    predictions_val = [np.sign(variational_classifier(weights, bias, f)) for f in feats_val]

    # Compute accuracy on train and validation set
    acc_train = accuracy(Y_train, predictions_train)
    acc_val = accuracy(Y_val, predictions_val)

    acc_test = test_accuracy(weights, bias)

    # print(
    #     "Iter: {:5d} | Cost: {:0.7f} | Acc train: {:0.7f} | Acc validation: {:0.7f} "
    #     "".format(it + 1, cost(weights, bias, features, Y), acc_train, acc_val)
    # )

    print(
        "Iter: {:5d} | Cost: {:0.7f} | Acc train: {:0.7f} | Acc validation: {:0.7f} | Acc test: {:0.7f} "
        "".format(it + 1, cost(weights, bias, features, Y), acc_train, acc_val, acc_test)
    )
    # if acc_train >= 0.88 and acc_val >= 0.88:
    if acc_train >= 0.95 and acc_val >= 0.95 and acc_test >= 0.95:
        # early stop
        break



Iter:     1 | Cost: 1.0466975 | Acc train: 0.4888889 | Acc validation: 0.4400000 | Acc test: 0.5166667 
Iter:     2 | Cost: 1.0348649 | Acc train: 0.4888889 | Acc validation: 0.4400000 | Acc test: 0.5166667 
Iter:     3 | Cost: 1.0252145 | Acc train: 0.4888889 | Acc validation: 0.4400000 | Acc test: 0.5166667 
Iter:     4 | Cost: 1.0199124 | Acc train: 0.4888889 | Acc validation: 0.4400000 | Acc test: 0.5166667 
Iter:     5 | Cost: 1.0150862 | Acc train: 0.4888889 | Acc validation: 0.4400000 | Acc test: 0.5166667 
Iter:     6 | Cost: 1.0047094 | Acc train: 0.4888889 | Acc validation: 0.4400000 | Acc test: 0.5166667 
Iter:     7 | Cost: 0.9987388 | Acc train: 0.4355556 | Acc validation: 0.4933333 | Acc test: 0.9333333 
Iter:     8 | Cost: 0.9933173 | Acc train: 0.5422222 | Acc validation: 0.6133333 | Acc test: 0.8583333 
Iter:     9 | Cost: 0.9871423 | Acc train: 0.6088889 | Acc validation: 0.6133333 | Acc test: 0.7916667 
Iter:    10 | Cost: 0.9784368 | Acc train: 0.6311111 | Acc valid

Load the test dataset and apply same transformations as we did with the train dataset.


In [8]:
print(
    "Iter: {:5d} | Cost: {:0.7f} | Acc train: {:0.7f} | Acc validation: {:0.7f} | Acc test: {:0.7f} "
    "".format(it + 1, cost(weights, bias, features, Y), acc_train, acc_val, acc_test)
)


print('step: ', it)
print('bias: ', bias)    
print('weights:\n', weights)
np.save('/_jupyter/QC/QOSF-challenge-md-2022/z-temp/z-displacement-phase-b-001.npy', bias, allow_pickle=True)
np.save('/_jupyter/QC/QOSF-challenge-md-2022/z-temp/z-displacement-phase-w-001.npy', weights, allow_pickle=True)



Iter:    95 | Cost: 0.8489172 | Acc train: 0.5377778 | Acc validation: 0.6133333 | Acc test: 0.4916667 
step:  94
bias:  -1.0439236273312837
weights:
 [[[ 1.12663592e-02 -1.07993151e-02 -1.14746865e-02]
  [ 6.39474346e-02 -4.92395719e-02  2.68528039e-01]
  [ 9.49420807e-03  8.75512414e-04 -1.22543552e-02]
  [ 8.44362976e-03 -1.00021535e-02 -1.54477110e-02]]

 [[ 1.18802979e-02  3.16942612e-03  9.20858824e-03]
  [ 2.72537824e-02 -5.11822845e-03  2.29035928e-01]
  [-1.03424284e-02  6.81594518e-03 -8.03409664e-03]
  [-6.89549778e-03 -4.55532504e-03  1.74791590e-04]]

 [[-3.53993911e-03 -1.37495129e-02 -6.43618403e-03]
  [-4.24025003e-02 -2.44602623e-02  1.88813030e-01]
  [-1.10438334e-02  5.21650793e-04 -7.39562996e-03]
  [ 1.54301460e-02 -1.29285691e-02  2.67050869e-03]]

 [[-3.92828182e-04 -1.16809350e-02  5.23276661e-03]
  [-3.18770988e-02  1.21537307e-02  2.17504474e-01]
  [ 2.16323595e-02  1.33652795e-02 -3.69181838e-03]
  [-2.39379178e-03  1.09965960e-02  6.55263731e-03]]

 [[ 6.401

In [9]:

acc_test = test_accuracy(weights, bias)
print('Accuracy on test data: {:0.4f}'.format(acc_test))


Accuracy on test data: 0.4917


In [10]:
# # We can plot the continuous output of the variational classifier for the first two dimensions of the data set.
# plt.figure()
# cm = plt.cm.RdBu

# # make data for decision regions
# xx, yy = np.meshgrid(np.linspace(0.0, 1.5, 300), np.linspace(0.0, 1.5, 300))
# X_grid = [np.array([x, y]) for x, y in zip(xx.flatten(), yy.flatten())]

# pd.DataFrame(xx).shape
# pd.DataFrame(yy).shape
# pd.DataFrame(X_grid).shape

In [11]:

# # # preprocess grid points like data inputs above
# # padding = 0.3 * np.ones((len(X_grid), 1))
# # X_grid = np.c_[np.c_[X_grid, padding], np.zeros((len(X_grid), 1))]  # pad each input

# # normalization = np.sqrt(np.sum(X_grid ** 2, -1))
# # X_grid = (X_grid.T / normalization).T  # normalize each input

# # features_grid = np.array(
# #     [get_angles(x) for x in X_grid]
# # )  # angles for state preparation are new features

# X_grid = X_norm
# features_grid = X_grid
# predictions_grid = [variational_classifier(weights, bias, f) for f in features_grid]
# # Z = np.reshape(predictions_grid, xx.shape)
# Z = predictions_grid 

# # plot decision regions
# cnt = plt.contourf(
#     xx, yy, Z, levels=np.arange(-1, 1.1, 0.1), cmap=cm, alpha=0.8, extend="both"
# )
# plt.contour(
#     xx, yy, Z, levels=[0.0], colors=("black",), linestyles=("--",), linewidths=(0.8,)
# )
# plt.colorbar(cnt, ticks=[-1, 0, 1])

# # plot data
# plt.scatter(
#     X_train[:, 0][Y_train == 1],
#     X_train[:, 1][Y_train == 1],
#     c="b",
#     marker="o",
#     edgecolors="k",
#     label="class 1 train",
# )
# plt.scatter(
#     X_val[:, 0][Y_val == 1],
#     X_val[:, 1][Y_val == 1],
#     c="b",
#     marker="^",
#     edgecolors="k",
#     label="class 1 validation",
# )
# # plt.scatter(
# #     X_train[:, 0][Y_train == -1],
# #     X_train[:, 1][Y_train == -1],
# #     c="r",
# #     marker="o",
# #     edgecolors="k",
# #     label="class -1 train",
# # )
# # plt.scatter(
# #     X_val[:, 0][Y_val == -1],
# #     X_val[:, 1][Y_val == -1],
# #     c="r",
# #     marker="^",
# #     edgecolors="k",
# #     label="class -1 validation",
# # )

# plt.legend()
# plt.show()